In [70]:
import pandas as pd
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, WheelZoomTool, MultiChoice
from bokeh.plotting import figure
from bokeh.tile_providers import get_provider, CARTODBPOSITRON
from bokeh.palettes import Category10_3
from bokeh.io import output_notebook
from pyproj import Proj, transform
from bokeh.models import DateRangeSlider, RangeSlider, HoverTool
from bokeh.layouts import column, row
from bokeh.models import CustomJS
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
from bokeh.models.widgets import DataTable, DateFormatter, StringFormatter, NumberFormatter, SelectEditor, StringEditor, IntEditor
from bokeh.models import DatePicker
from bokeh.layouts import column
from bokeh.plotting import curdoc
from datetime import date
from bokeh.models import DatePicker, MultiChoice, CustomJS
from bokeh.layouts import column
from bokeh.plotting import curdoc
from bokeh.models.widgets import StringFormatter
from bokeh.models import DataTable, DateFormatter, NumberFormatter, StringFormatter, StringEditor, NumberEditor, SelectEditor
from datetime import date
from bokeh.models import HoverTool, WheelZoomTool, RangeTool
from bokeh.plotting import figure
from bokeh.tile_providers import get_provider, CARTODBPOSITRON
from bokeh.palettes import Category10_3
from bokeh.models import Toggle, Select, Div
from bokeh.layouts import column
from bokeh.plotting import curdoc
from bokeh.models.widgets import StringFormatter
from bokeh.models import DataTable, DateFormatter, NumberFormatter, StringFormatter, StringEditor, NumberEditor, SelectEditor
from datetime import date

In [46]:
df_all = pd.read_csv('data/df_all.csv')

In [47]:
# Breedtegraad en lengtegraad naar Mercator-projectie transformeren
in_proj = Proj(init='epsg:4326')
out_proj = Proj(init='epsg:3857')
df_all['mercator_x'], df_all['mercator_y'] = transform(in_proj, out_proj, df_all['longitude'].values, df_all['latitude'].values)

C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\ProgramData\Anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\delta\AppData\Local\Temp/ipykernel_6232/3089975609.py:4: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  df_all['mercat

In [48]:
# Gegevensbron maken
source = ColumnDataSource(df_all)

In [49]:
# Definieer de kolommen van de tabel met de juiste formatters
columns = [
    TableColumn(field="Datum", title="Datum"),
    TableColumn(field="dag", title="Dag"),
    TableColumn(field="Plaats", title="Plaats", formatter=StringFormatter(font_style="bold")),
    TableColumn(field="Klassementen", title="Klassementen", formatter=StringFormatter(font_style="italic")),
    TableColumn(field="Kwalificatie", title="Kwalificatie"),
    TableColumn(field="Afstanden(km)", title="Afstanden(km)"),
    TableColumn(field="min.leeftijd", title="Min. leeftijd"),
    TableColumn(field="Organisator", title="Organisator"),
    TableColumn(field="gemeente", title="Gemeente"),
    TableColumn(field="Opmerking", title="Opmerking")
]

# Maak de DataTable-widget met de gegevensbron en kolommen
data_table = DataTable(source=source, columns=columns, editable=False, width_policy="fit", index_position=None)

# Kopieer de oorspronkelijke tabelgegevens
original_data = {key: value.copy() for key, value in source.data.items()}

In [50]:
# De kaart van Nederland weergeven vanaf de startpositie
x_range = (422075.1311397397, 766958.9312551429)
y_range = (6574807.400878754, 7071342.360802666)

# Tile Provider configureren
tile_provider = get_provider(CARTODBPOSITRON)

# Plot aanmaken
p = figure(x_range=x_range, y_range=y_range,
           x_axis_type="mercator", y_axis_type="mercator",
           title="Survivalrun Events")

# Cirkels toevoegen aan de kaart
circle_renderer = p.circle(x="mercator_x", y="mercator_y", size=10, fill_color=Category10_3[0], fill_alpha=0.6, source=source)

# Tegels (kaart) toevoegen
p.add_tile(tile_provider)

p.toolbar.active_scroll = p.select_one(WheelZoomTool)

# Voeg de HoverTool toe aan de plot en koppel deze aan de Circle-renderer
hover_tool = HoverTool(renderers=[circle_renderer],
                       tooltips=[("Plaats", "@Plaats"),
                                 ("Dag", "@dag"),
                                 ("Datum", "@Datum"),
                                 ("Klassementen", "@Klassementen"),
                                 ("Kwalificatie", "@Kwalificatie"),
                                 ("Afstanden(km)", "@{Afstanden(km)}"),
                                 ("Min. leeftijd", "@{min.leeftijd}"),
                                 ("Organisator", "@Organisator"),
                                 ("Gemeente", "@gemeente"),
                                 ("Opmerking", "@Opmerking"),])
p.add_tools(hover_tool)

In [51]:
# Maak een lijst van unieke waarden voor 'dag'
unique_days = list(set(df_all['dag']))

# Maak de MultiChoice-widget voor 'dag'
day_multi_choice = MultiChoice(title="Dag", value=unique_days, options=unique_days)

# Callback-functie voor het bijwerken van de tabel
callback = CustomJS(args=dict(source=source, day_multi_choice=day_multi_choice, original_data=original_data), code="""
    const data = source.data;
    const days = data['dag'];

    const selected_days = day_multi_choice.value;

    // Filteren op dagen
    const indices = [];
    for (let i = 0; i < days.length; i++) {
        const day = days[i];

        if (selected_days.includes(day)) {
            indices.push(i);
        }
    }

    // Bijwerken van de tabelgegevens
    if (indices.length > 0) {
        const filtered_data = {};
        for (const key in original_data) {
            filtered_data[key] = original_data[key].filter((_, index) => indices.includes(index));
        }
        source.data = filtered_data;
    } else {
        source.data = original_data;
    }
""")

# Koppel de callback-functie aan de `MultiChoice`-widget
day_multi_choice.js_on_change('value', callback)

In [71]:
# Maak de Select-widget voor 'Kwalificatie'
kwalificatie_select = Select(title="Kwalificatie:", options=["Alle wedstrijden", "Alleen Kwalificatie"], value="Alle wedstrijden")

# Callback-functie voor het bijwerken van de tabel en de dropdown-stijl
callback = CustomJS(args=dict(source=source, kwalificatie_select=kwalificatie_select, original_data=original_data), code="""
    const data = source.data;
    const kwalificaties = data['Kwalificatie'];

    const selected_kwalificatie = kwalificatie_select.value === "Alle wedstrijden" ? null : "Ja";

    // Filteren op Kwalificatie
    const indices = [];
    for (let i = 0; i < kwalificaties.length; i++) {
        const kwalificatie = kwalificaties[i];

        if (kwalificatie === selected_kwalificatie) {
            indices.push(i);
        }
    }

    // Bijwerken van de tabelgegevens
    if (indices.length > 0) {
        const filtered_data = {};
        for (const key in original_data) {
            filtered_data[key] = original_data[key].filter((_, index) => indices.includes(index));
        }
        source.data = filtered_data;
    } else {
        source.data = original_data;
    }
""")



# Koppel de callback-functie aan de Select-widget
kwalificatie_select.js_on_change('value', callback)

In [64]:
# Maak twee DatePicker-widgets voor de start- en einddatum
start_date_picker = DatePicker(title="Startdatum", value=min(df_all['Datum_dt']), min_date=min(df_all['Datum_dt']), max_date=max(df_all['Datum_dt']))
end_date_picker = DatePicker(title="Einddatum", value=max(df_all['Datum_dt']), min_date=min(df_all['Datum_dt']), max_date=max(df_all['Datum_dt']))

# Callback-functie voor het bijwerken van de tabel
callback = CustomJS(args=dict(source=source, start_date_picker=start_date_picker, end_date_picker=end_date_picker, original_data=original_data), code="""
    const data = source.data;
    const dates = data['Datum_dt'];

    const selected_start_date = new Date(start_date_picker.value);
    const selected_end_date = new Date(end_date_picker.value);

    // Filteren op datum
    const indices = [];
    for (let i = 0; i < dates.length; i++) {
        const date = new Date(dates[i]);

        if (date >= selected_start_date && date <= selected_end_date) {
            indices.push(i);
        }
    }

    // Bijwerken van de tabelgegevens
    if (indices.length > 0) {
        const filtered_data = {};
        for (const key in original_data) {
            filtered_data[key] = original_data[key].filter((_, index) => indices.includes(index));
        }
        source.data = filtered_data;
    } else {
        source.data = original_data;
    }
""")

# Koppel de callback-functie aan de `DatePicker`-widgets
start_date_picker.js_on_change('value', callback)
end_date_picker.js_on_change('value', callback)

In [72]:
# HTML-bestand genereren
output_file("docs\index.html")

layout = column(start_date_picker,end_date_picker,day_multi_choice,kwalificatie_select, p, data_table)

# Het dashboard weergeven
show(layout)